In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FullyConnectedNetwork(nn.Module):
    def __init__(self):
        super(FullyConnectedNetwork, self).__init__()
        
        # First fully connected layer
        self.fc1 = nn.Linear(475, 512)
        self.dropout1 = nn.Dropout(0.5)

        # Second fully connected layer
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)

        # Third fully connected layer
        self.fc3 = nn.Linear(256, 128)

        # Output layer
        self.fc4 = nn.Linear(128, 15)

    def forward(self, x):
        # Flatten the image input
        x = x.view(x.size(0), -1)

        # Pass through the first layer, apply ReLU and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        # Second layer
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        # Third layer
        x = F.relu(self.fc3(x))

        # Output layer with softmax activation
        x = self.fc4(x)
        x = F.log_softmax(x, dim=1)
        return x

# Create the neural network
model = FullyConnectedNetwork()

# Print the model structure
print(model)


FullyConnectedNetwork(
  (fc1): Linear(in_features=475, out_features=512, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=15, bias=True)
)
